In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
import os
import time
import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Dropout, Flatten, merge
from keras.layers import LSTM, Convolution1D, MaxPooling1D
from keras.layers.core import Activation
from keras.optimizers import SGD
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.regularizers import l2
from keras import callbacks as cb
from scipy import signal

Using TensorFlow backend.


In [2]:
from __future__ import print_function
from __future__ import absolute_import

import sys
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Permute
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.layers.recurrent import GRU
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model

sys.path.insert(0, './deep-learning-models')
from audio_conv_utils import decode_predictions, preprocess_input

TH_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.3/music_tagger_crnn_weights_tf_kernels_th_dim_ordering.h5'
TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.3/music_tagger_crnn_weights_tf_kernels_tf_dim_ordering.h5'

#K._LEARNING_PHASE = tf.constant(0)
#K.LEARNING_PHASE = tf.constant(0)
#K.set_learning_phase(0)

In [3]:
from __future__ import print_function

import os
import sys
import random
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, merge
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers.core import Masking
from keras.layers.recurrent import LSTM
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger
from keras.optimizers import rmsprop
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

sys.path.insert(0, './deep-learning-models')
from music_tagger_crnn import MusicTaggerCRNN
from audio_conv_utils import preprocess_input, decode_predictions

from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import pylab as P

import socket
if socket.gethostname()=="euler":
    os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [4]:
def create_run_name(params):
    k_v_names = []
    for k,v in params.items():
        k_v_names.append((k,v))
        
    k_v_names.sort(key=lambda x: x[0])
    out_str = ""
    for k,v in k_v_names:
        out_str += k + "_" + str(v).replace('.', '-') + "|"
    return out_str[:-1]

In [5]:
glob_margin = 1
test = False
params = {}
params['glob_margin'] = 1.0
params['num_hidden'] = 128
params['w_reg'] = 0.001
params['u_reg'] = 0.001
params['return_sequences'] = False

run_name = create_run_name(params)
run_name
# save every few epochs, set num_epochs to infinity
# save training and validation loss to a logfile

'glob_margin_1-0|num_hidden_128|return_sequences_False|u_reg_0-001|w_reg_0-001'

In [6]:
# load all necessary files
valids = pickle.load(open("valids.pkl", "rb"))
pairs = pickle.load(open("pairs.pkl", "rb"))
msd_to_info = pickle.load(open("msd_to_info.pkl", "rb"))
print("valids size:", sys.getsizeof(valids)/1000000, "mB")
print("pairs size:", sys.getsizeof(pairs)/1000000, "mB")
print("msd_to_info size:", sys.getsizeof(msd_to_info)/1000000, "mB")

valids size: 2.097376 mB
pairs size: 2.853112 mB
msd_to_info size: 1.57296 mB


In [7]:
"""
def euc_dist(x):
    'Merge function: euclidean_distance(u,v)'
    diff = tf.subtract(x[0], x[1])
    diff_sq = tf.square(diff)
    #output = tf.reduce_sum(diff_sq, 2)
    output = tf.reduce_sum(diff_sq, 1)
    output = tf.reshape(output, (-1,1))
    print("output.get_shape()", output.get_shape())
    return output
"""

def euc_dist(x):
    'Merge function: euclidean_distance(u,v)'
    print(x[0].get_shape(), x[1].get_shape())
    diff = tf.subtract(x[0], x[1])
    diff_sq = tf.square(diff)
    #output = tf.reduce_sum(diff_sq, 2)
    output_sq = tf.reduce_sum(diff_sq, 1)
    output = tf.sqrt(output_sq)
    output = tf.reshape(output, (-1,1))
    print("output.get_shape()", output.get_shape())
    return output

def custom_crossentropy(y_true, y_pred):
    dist = y_pred
    dist_g0 = K.maximum(dist, 0)
    return K.mean(y_true * K.log(K.tanh(dist_g0)) + (1 - y_true) * K.log(1 - K.tanh(dist_g0)))

def euc_dist_shape(input_shape):
    'Merge output shape'
    shape = list(input_shape)
    #outshape = (shape[0][0],shape[0][1])
    outshape = (shape[0][0],1)
    print("outshape", outshape)
    return tuple(outshape)

# add callback for learning rate
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 150.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

"""#train MMD net
optimizer = keras.optimizers.rmsprop(lr=0.0)

calibMMDNet.compile(optimizer=optimizer, loss=lambda y_true,y_pred: 
               cf.MMD(block3_output,target,MMDTargetValidation_split=0.1).KerasCost(y_true,y_pred))
K.get_session().run(tf.global_variables_initializer())

sourceLabels = np.zeros(source.shape[0])
calibMMDNet.fit(source,sourceLabels,nb_epoch=500,batch_size=1000,validation_split=0.1,verbose=1,
           callbacks=[lrate, mn.monitorMMD(source, target, calibMMDNet.predict),
                      cb.EarlyStopping(monitor='val_loss',patience=50,mode='auto')])"""

"#train MMD net\noptimizer = keras.optimizers.rmsprop(lr=0.0)\n\ncalibMMDNet.compile(optimizer=optimizer, loss=lambda y_true,y_pred: \n               cf.MMD(block3_output,target,MMDTargetValidation_split=0.1).KerasCost(y_true,y_pred))\nK.get_session().run(tf.global_variables_initializer())\n\nsourceLabels = np.zeros(source.shape[0])\ncalibMMDNet.fit(source,sourceLabels,nb_epoch=500,batch_size=1000,validation_split=0.1,verbose=1,\n           callbacks=[lrate, mn.monitorMMD(source, target, calibMMDNet.predict),\n                      cb.EarlyStopping(monitor='val_loss',patience=50,mode='auto')])"

In [8]:
def euc(vects):
    x, y = vects
    return K.sum(K.square(x - y), axis=1, keepdims=True)

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = glob_margin
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def siamese_accuracy(y_true, y_pred):
    margin = tf.constant([glob_margin], dtype='float32')
    zero = tf.constant([0.0])
    print(y_pred.get_shape())
    return K.mean(K.equal(np.abs(1 - y_true), K.cast(K.greater_equal(euc([y_pred, zero]), margin), 'float32')))

In [9]:
class SimplestDemoDataHandler:
    def __init__(self, loss='binary_crossentropy', timesteps=1, num_vals=2, return_sequences=True):
        self.loss = loss
        if loss == 'binary_crossentropy':
            self.true_y = 0.5
            self.false_y = 1.0
        elif loss == 'contrastive_loss':
            self.true_y = 1.0
            self.false_y = 0.0
        self.timesteps = timesteps
        self.num_vals = num_vals
        self.return_sequences = return_sequences


    def get_mixed(self, num_pairs):
        num_pairs = int(num_pairs)
        num_examples = int(num_pairs * 2)
        x_feed1 = np.empty((num_examples, self.timesteps, 1))
        x_feed2 = np.empty((num_examples, self.timesteps, 1))
        if self.return_sequences:
            y = np.empty((num_examples, self.timesteps))
        else:
            y = np.empty((num_examples))
        cur_examples = 0

        # grab batches
        x_false_aud, x_false_vid, x_false_lab = self._get_false(num_pairs)
        x_true_aud, x_true_vid, x_true_lab = self._get_true(num_pairs)
        # put them together
        x_aud = np.concatenate((x_false_aud, x_true_aud), axis=0)
        x_vid = np.concatenate((x_false_vid, x_true_vid), axis=0)
        y = np.concatenate((x_false_lab, x_true_lab), axis=0)
        # shuffle them
        rand_state = np.random.get_state()
        np.random.shuffle(x_aud)
        np.random.set_state(rand_state)
        np.random.shuffle(x_vid)
        np.random.set_state(rand_state)
        np.random.shuffle(y)
        # add them to our main numpy arrays
        x_feed1[:,:,:] = x_aud
        x_feed2[:,:,:] = x_vid
        if self.return_sequences:
            y[:,:] = y
        else:
            y[:] = y

        return x_feed1, x_feed2, y

    def _get_true(self, num_examples):
        same = np.random.randint(0,self.num_vals,(num_examples, self.timesteps, 1))
        x_feed1 = same
        x_feed2 = same.copy()
        if self.return_sequences:
            y = np.full((num_examples, self.timesteps), self.true_y)
        else:
            y = np.full((num_examples), self.true_y)

        return x_feed1, x_feed2, y

    def _get_false(self, num_examples):
        x_feed1 = np.random.randint(0,self.num_vals,(num_examples, self.timesteps, 1))
        if self.timesteps < 4:
            x_feed2 = np.empty((num_examples, self.timesteps, 1))
            for i in range(num_examples):
                x_feed2[i,:,:] = np.random.randint(0, self.num_vals, (self.timesteps, 1))
                #print(x_feed1[i,:,:].shape)
                #print(x_feed1[i,:,:])
                #print(np.isclose(x_feed2[i,:,:], x_feed1[i,:,:]).all())
                while np.isclose(x_feed2[i,:,:], x_feed1[i,:,:]).all():
                    x_feed2[i,:,:] = np.random.randint(0, self.num_vals, (self.timesteps, 1))
        else:
            x_feed2 = np.random.randint(0,self.num_vals,(num_examples, self.timesteps, 1))
        if self.return_sequences:
            y = np.full((num_examples, self.timesteps), self.false_y)
        else:
            y = np.full((num_examples), self.false_y)

        return x_feed1, x_feed2, y

In [10]:
def tied_weights(num_hidden, w_reg, u_reg, return_sequences):
    lstm_1 = LSTM(num_hidden, W_regularizer=l2(w_reg), U_regularizer=l2(u_reg), return_sequences=return_sequences)
    lstm_2 = lstm_1
    return lstm_1, lstm_2

def diff_weights(num_hidden, w_reg, u_reg, return_sequences):
    lstm_1 = LSTM(num_hidden, W_regularizer=l2(w_reg), U_regularizer=l2(u_reg), return_sequences=return_sequences)
    lstm_2 = LSTM(num_hidden, W_regularizer=l2(w_reg), U_regularizer=l2(u_reg), return_sequences=return_sequences)
    return lstm_1, lstm_2

In [11]:
return_sequences = False
loss = 'contrastive_loss'
#weight_type = 'diff_weights'
weight_type = 'tied_weights'
timesteps = 100
num_hidden = 16
num_vals = 2
w_reg = 0.0001
u_reg = 0.0001
x_shape = (timesteps, 1)

input_1 = Input(shape=x_shape, name='input_1')
input_2 = Input(shape=x_shape, name='input_2')

if weight_type == 'tied_weights':
    lstm_1, lstm_2 = tied_weights(num_hidden, w_reg, u_reg, return_sequences)
else:
    lstm_1, lstm_2 = diff_weights(num_hidden, w_reg, u_reg, return_sequences)

processed_1 = lstm_1(input_1)
processed_2 = lstm_2(input_2)

# merged
#merged = merge([processed_1, processed_2], mode=euc_dist, output_shape=euc_dist_shape)
merged = merge([processed_1, processed_2], mode=euc, output_shape=eucl_dist_output_shape)
#merged = merge([processed_1, processed_2], mode='concat')
output = merged
#output = merge([lstm_aud, lstm_vid], mode=euclidean_distance, output_shape=eucl_dist_output_shape)

#output = Activation('sigmoid')(output)

#output = Dense(1, activation='sigmoid')(merged)

print("output: " + str(output.get_shape()))

model = Model(input=[input_1, input_2], output=[output])
adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
if loss == 'binary_crossentropy':
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy', 'mean_squared_error'])
else:
    model.compile(optimizer=adam, loss=contrastive_loss, metrics=[siamese_accuracy, 'mean_squared_error'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, return_sequences=False, recurrent_regularizer=<keras.reg..., kernel_regularizer=<keras.reg...)`
  


output: (?, 1)
(?, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [12]:
adam = keras.optimizers.Adam(lr=1e-5*2)
model.compile(optimizer=adam, loss=contrastive_loss, metrics=[siamese_accuracy, 'mean_squared_error'])

(?, 1)


In [13]:
#sddh = SimplestDemoDataHandler(timesteps=10, loss='binary_crossentropy')
sddh = SimplestDemoDataHandler(timesteps=timesteps, num_vals=num_vals, loss=loss, return_sequences=return_sequences)
lstm_1_f = K.function((model.inputs[0],), [processed_1])
lstm_2_f = K.function((model.inputs[1],), [processed_2])
merged_f = K.function(model.inputs, [merged])
output_f = K.function(model.inputs, [output])

In [14]:
batch_size = 32
num_epochs = 10000
num_train = 1024
num_val = 1024
print_our_acc = False
how_verbose = 1
for epoch in range(num_epochs):
    x_1_train, x_2_train, y_train = sddh.get_mixed(num_train / 2)
    x_1_val, x_2_val, y_val = sddh.get_mixed(num_val / 2)
    #x_1_val, x_2_val, y_val = x_1_train, x_2_train, y_train
    print("real epochs: " + str(epoch))
    if epoch % how_verbose == 0:
        verbosity = 1
        if print_our_acc:
            
            verbosity = 1
            yhat = model.predict([x_1_val, x_2_val])
            num_correct = 0
            num_print = 0
            num_over = 0
            print("glob_margin", glob_margin)
            for dist, y_v in zip(yhat, y_val):
                dist = np.linalg.norm(dist)
                if dist > glob_margin:
                    num_over += 1
                #if (dist > 0.75 and y_v == 1) or (dist <= 0.75 and y_v == 0.5):
                #if (dist[-1] > 0.75 and y_v[-1] == 1) or (dist[-1] <= 0.75 and y_v[-1] == 0.5):
                if (dist >= glob_margin and y_v[-1] == 0.0) or (dist < glob_margin and y_v[-1] == 1.0):
                    if num_print < 10:
                        print(dist, "|", y_v[-1], "|", "correct")
                #if (dist > glob_margin and y_v == 1.0) or (dist <= glob_margin and y_v == 0.0):
                #if (dist > 0.75 and y_v == 1.0) or (dist <= 0.75 and y_v == 0.5):
                    #print("correct -- dist: " + str(dist[-1]) + ", y_v: " + str(y_v[-1]))
                    num_correct += 1
                else:
                    if num_print < 10:
                        print(dist, "|", y_v[-1], "|", "incorrect")
                    pass
                num_print += 1
                    #print("not_correct -- dist: " + str(dist[-1]) + ", y_v: " + str(y_v[-1]))
            #print([(x,y) for x,y in zip(yhat[:10], y_val[:10])])
            #for x,y in zip(yhat[:10], y_val[:10]):
                #print(x,y)
            print(num_over)
            print(num_correct / num_val)
    
    model.fit([x_1_train, x_2_train], [y_train], 
              validation_data=([x_1_val, x_2_val], [y_val]),
              nb_epoch=1, batch_size=batch_size, verbose=verbosity)
    
    if epoch % how_verbose == 0:
        verbosity = 0
        
#callbacks=(lrate, mn.monitorMMD(source, target, calibMMDNet.predict),
                      #cb.EarlyStopping(monitor='val_loss',patience=50,mode='auto'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:49: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


real epochs: 0
Train on 1024 samples, validate on 1024 samples
Epoch 1/1
1024/1024 [==============================] - 6s - loss: 0.4867 - siamese_accuracy: 0.5000 - mean_squared_error: 0.5003 - val_loss: 0.4864 - val_siamese_accuracy: 0.5000 - val_mean_squared_error: 0.5003
real epochs: 1
Train on 1024 samples, validate on 1024 samples
Epoch 1/1
1024/1024 [==============================] - 6s - loss: 0.4852 - siamese_accuracy: 0.5000 - mean_squared_error: 0.5003 - val_loss: 0.4868 - val_siamese_accuracy: 0.5000 - val_mean_squared_error: 0.5003
real epochs: 2
Train on 1024 samples, validate on 1024 samples
Epoch 1/1


KeyboardInterrupt: 

In [ ]:
x_1, x_2, y = sddh.get_mixed(8)
hist = model.fit([x_1, x_2], y, nb_epoch=1, batch_size=1)
l_1 = lstm_1_f([x_1])
l_2 = lstm_2_f([x_2])
m = merged_f([x_1, x_2])
o = output_f([x_1, x_2])

In [ ]:
hist.history

In [ ]:
y

In [ ]:
for x1,x2,y_t in zip(x_1, x_2, y):
    print("x1", x1, "x2", x2, "y_t", y_t)

In [ ]:
for l1,l2,y_t in zip(l_1[0],l_2[0],y):
    #print("l1",l1, "l2",l2, "dist",np.abs(l1-l2), "y_t",y_t)
    print("dist",np.abs(np.linalg.norm(l1-l2)), "y_t",y_t)

In [ ]:
o[0].mean()

In [ ]:
dist = model.predict([x_1, x_2])

In [ ]:
dist

In [ ]:
sddh.audio.shape[0] / 1000 / 1000

In [ ]:
model.layers

In [ ]:
model.layers[2].get_weights()